In [1]:
import pandas as pd
import numpy as np

In [2]:
path_to_file = "/Users/aditgupta/Google Drive/Research/T2D/Summer_2018/drug_names.tsv" #this is the path to the file for all the drug names and corresponding ID's
df5 = pd.read_csv(path_to_file,delimiter="\t", header=None) #reads file, converts to csv, and explains to fucntio that the delimeter are tabs
df5.rename(columns={0:'ID'}, inplace=True) #renames index zero to 'ID'
df5.rename(columns={1:'drug_name'}, inplace=True) #renames index one to drug name
#OVERALL: df5 contains the drug name based on the 'CID' ID 

In [3]:
import xml.etree.ElementTree as ET
import lxml.html
#importing libraries that parse the xml file with detailed drug information 

In [4]:
path_to_file = '/Users/aditgupta/Google Drive/Research/T2D/Summer_2018/drugbank_full_database.xml' #file location of the drugbank database

In [5]:
tree = ET.parse(path_to_file) # function that parses the xml file from drugbank

In [6]:
def find_categories_matching_string(drug_element, category_name):
    all_categories = drug_element.findall(".//*{http://www.drugbank.ca}category")
    categories_wanted = list(filter(lambda category: category.text == category_name, all_categories))
    return categories_wanted

In [7]:
all_drugs = tree.findall("{http://www.drugbank.ca}drug")

In [8]:
def get_id_and_name_from_target(target):
    id = target.find("{http://www.drugbank.ca}id").text
    name = target.find("{http://www.drugbank.ca}name").text
    effect = target.find("*{http://www.drugbank.ca}category") #Blood Glucose Lowering
    gene = target.find("*{http://www.drugbank.ca}gene-name")
    #glucose_categories = find_categories_matching_string(target, "Blood Glucose Lowering Agents")
    #diabetes_categories = find_categories_matching_string(target, "Drugs Used in Diabetes")
    if gene is None:
        gene_found = None
        
    else:
        gene_found= gene.text
    return id, name, gene_found
#function that returns the target id, target name, and gene name of a particular sample

In [9]:
new_results = []
total_found = 0
for drug in all_drugs:
    drug_name = drug.find("{http://www.drugbank.ca}name").text
    drug_id = drug.find("{http://www.drugbank.ca}drugbank-id[@primary]").text
    element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
    list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
    bp_categories = find_categories_matching_string(drug, "Antihypertensive Agents")
    if  bp_categories == []:
        bp_categories = None
    for target in list_of_targets:
        target_id, target_name, target_gene = get_id_and_name_from_target(target)
        new_results.append([drug_name, drug_id, target_id, target_name, target_gene, bp_categories])
    total_found += 1
#creates an array and list that contains the drug name, drug id, target id, target name, and target gene as extracted from drug bank

In [10]:
drug = all_drugs[0]
element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
target = list_of_targets[0]

In [11]:
second_results = []
total_found = 0
for drug in all_drugs:
    drug_name = drug.find("{http://www.drugbank.ca}name").text
    drug_id = drug.find("{http://www.drugbank.ca}drugbank-id[@primary]").text
    element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
    list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
    for target in list_of_targets:
        target_id, target_name, target_gene = get_id_and_name_from_target(target)
        second_results.append([drug_name, drug_id, target_id, target_name, target_gene])
    total_found += 1
#creates an array and list that contains the drug name, drug id, target id, target name, and target gene as extracted from drug bank

In [12]:
df = pd.DataFrame(new_results) #df contains the results from the parsing of the xml file from drugbank
df.columns = ["drug_name", "drug_id", "target_id", "target_name", "target_gene","bp",] #renaming the columns of the dataframe

In [13]:
df_category1 = df.dropna(subset = ['bp'])
df_category1.head()

,drug_name,drug_id,target_id,target_name,target_gene,bp
1290,Valsartan,DB00177,BE0000062,Type-1 angiotensin II receptor,AGTR1,[[]]
1291,Ramipril,DB00178,BE0000221,Angiotensin-converting enzyme,ACE,[[]]
1292,Ramipril,DB00178,BE0005831,B1 bradykinin receptor,BDKRB1,[[]]
1328,Esmolol,DB00187,BE0000172,Beta-1 adrenergic receptor,ADRB1,[[]]
1353,Betaxolol,DB00195,BE0000172,Beta-1 adrenergic receptor,ADRB1,[[]]


In [14]:
df3 = df['drug_name'].value_counts() 
df4 = df3.to_frame(name=None)
df5 = df4.reset_index()
df6 = df5.rename(columns={'drug_name':'number_of_targets'}, inplace=False)
df7 = df6.rename(columns={'index':'drug_name'}, inplace=False)
df7.head()

,drug_name,number_of_targets
0,Fostamatinib,300
1,Artenimol,192
2,Copper,147
3,NADH,144
4,Zinc,124


In [15]:
target_freq = df['target_id'].value_counts() #number of drugs that target a given target ID in a series
df8 = target_freq.to_frame(name=None)
df9 = df8.reset_index()
df10 = df9.rename(columns={'target_id':'target_freq'}, inplace=False)
df11 = df10.rename(columns={'index':'target_id'}, inplace=False)
df11.head(10)

,target_id,target_freq
0,BE0001072,137
1,BE0000048,112
2,BE0000123,111
3,BE0000756,108
4,BE0000451,105
5,BE0000442,104
6,BE0004796,101
7,BE0000322,100
8,BE0001739,99
9,BE0000092,97


In [16]:
df12 = df7[(df7['number_of_targets'])<3]  
df12.head()

,drug_name,number_of_targets
1508,Aminomethylcyclohexane,2
1509,IGN311,2
1510,S-Methylcysteine,2
1511,Irosustat,2
1512,Veliparib,2


In [17]:
df13 = df11[(df11['target_freq'])>1]
df13.head()

,target_id,target_freq
0,BE0001072,137
1,BE0000048,112
2,BE0000123,111
3,BE0000756,108
4,BE0000451,105


In [18]:
df14 = pd.merge(df_category1, df12, on='drug_name', how='inner')
df15 = df14.dropna(subset= ['target_gene'])
df16 = pd.merge(df15, df13, on='target_id', how='outer')  
df17 = df16.dropna()
df17['target_gene'].unique()
#all cleaned genes for 

array(['AGTR1', 'ACE', 'BDKRB1', 'ADRB1', 'ADRB2', 'REN', 'SLC12A1',
       'SLC6A2', 'PTGFR', 'NPR1', 'CACNG1', 'SLC12A3', 'EDNRA', 'EDNRB',
       'MME', 'ADRA2A', 'ADRA2B', 'ACE2', 'TH', 'SFRP4', 'PPARG', 'DDC',
       'JUN', 'SLC18A2', 'CHRNA10', 'ATP1A1', 'AOC3', 'P4HA1', 'MAOB',
       'MAOA', 'GUCY1A2', 'ADRA1A', 'CHRNA2', 'ABCC9', 'CACNA1I',
       'CACNA1C', 'CACNA1G', 'NISCH', 'PTGIR', 'DRD1', 'DRD2', 'HTR2A',
       'ABCB1'], dtype=object)